# SensRay MeshEarthModel Demo

This demo showcases the core features of `MeshEarthModel`:
- Build a tetrahedral sphere mesh
- Map properties from a 1D Earth model (e.g., PREM) onto mesh cells
- Visualize constant-per-cell values on a spherical shell and on great-circle slices
- Compute per-cell path lengths for a synthetic ray and derive sensitivity kernels
- Overlay points (e.g., source and receiver) on plots

In [1]:
import numpy as np
from sensray.mesh.earth_model import MeshEarthModel
from sensray.core.ray_paths import RayPathTracer

## 1) Create a tetrahedral sphere mesh
We'll generate a reasonably coarse sphere to keep things fast in a notebook.

In [2]:
# Adjust mesh_size_km to trade accuracy vs speed
mesh_model = MeshEarthModel.from_pygmsh_sphere(mesh_size_km=500)
mesh_model.radius_km

6371.0

## 2) Visualize the empty mesh

Initially there are no values associated with the mesh, but it can still be visualzied. We can visualize either a spherical surface cutting through the mesh, or a great circle slice cutting through the mesh.

In [3]:
# Sphere cut through the mesh at radius 5000 km
mesh_model.plot_sphere(radius_km=5000, wireframe=True).show()

# Slice cut through the mesh along a great circle. We specify the endpoints of
# the great circle even though here they are not actually associated with a
# source or a receiver
mesh_model.plot_slice(source_lat=0, source_lon=0, receiver_lat=45, receiver_lon=90).show()

## 3) Map 1D Earth model properties onto mesh cells
We'll add S-wave and P-wave velocities (`vs`, `vp`) from the selected 1D model.

In [4]:
model_name = 'prem'  # TauP 1D model name
mesh_model.add_scalars_from_1d_model(model_name, properties=('vs','vp'), where='cell')
# Quick sanity check: show available cell_data keys
list(mesh_model.mesh.cell_data.keys())

# Sphere cut through the mesh at radius 5000 km, now colored by vs
mesh_model.plot_sphere(radius_km=5000, scalar_name='vs', wireframe=True).show()

## 4) Great-circle slice with constant-per-cell coloring
Polygons on the slice inherit their parent cell values for crisp, non-interpolated coloring.

In [5]:
source_lat, source_lon, source_depth = 0.0, 0.0, 10.0
receiver_lat, receiver_lon = 0.0, 80.0
p = mesh_model.plot_slice(
    source_lat=source_lat,
    source_lon=source_lon,
    receiver_lat=receiver_lat,
    receiver_lon=receiver_lon,
    scalar_name='vs',
    cmap='RdBu',
    wireframe=False
)
p.show()

## 5) Compute a synthetic ray, path lengths, and sensitivity kernel
We'll create a straight chord ray between two near-surface points and compute per-cell path lengths and a `K_vs` kernel.
Note: This is a synthetic example; real rays would typically be curved.

In [6]:
# Build a ray path between the source and receiver for a certain 1d model
tracer = RayPathTracer(model_name=model_name)
rays, info = tracer.get_ray_paths(source_lat=source_lat, source_lon=source_lon, source_depth=source_depth,
                            receiver_lat=receiver_lat, receiver_lon=receiver_lon, phases=['P'])
rays_coords = tracer.extract_ray_coordinates(rays)

# Select a single arrival (e.g., 'P'). Now you can pass the dict directly
ray_dict = rays_coords['P']

# Compute per-cell path lengths
L = mesh_model.compute_ray_cell_path_lengths(ray_dict, attach_name='ray_lengths', tol=1e-6)

# Compute sensitivity kernel K_vs = -L / (vs^2 + epsilon)
K_vs = mesh_model.compute_sensitivity_kernel(ray_dict, property_name='vs', attach_name='K_vs', epsilon=1e-6)

# Quick summaries
float(L.sum()), int(np.count_nonzero(L)), float(np.nanmax(np.abs(K_vs)))

(8412.542535123857, 49, 15.585394721785196)

## 6) Visualize the kernel on a slice and on a shell
Because kernels are stored as cell scalars, the constant-per-cell rendering applies here too.

In [9]:
p = mesh_model.plot_slice(
    source_lat=source_lat,
    source_lon=source_lon,
    receiver_lat=receiver_lat,
    receiver_lon=receiver_lon,
    scalar_name='K_vs',
    cmap='viridis',
    wireframe=True,
)
# Overlay the ray path on top of the slice
mesh_model.add_polyline(p, ray_dict, color='yellow', line_width=4.0)
p.show()

mesh_model.plot_sphere(radius_km=5000, scalar_name='K_vs', wireframe=True).show()

## 7) Overlay source and receiver points on a slice
The `add_points` helper accepts dict inputs with geographic coordinates and depth.

In [8]:
source_depth_km = 30.0
pl = mesh_model.plot_slice(
    source_lat=source_lat,
    source_lon=source_lon,
    receiver_lat=receiver_lat,
    receiver_lon=receiver_lon,
    scalar_name='vs',
    cmap='RdBu',
    wireframe=True,
)
mesh_model.add_points(pl, {
    'lat': [source_lat, receiver_lat],
    'lon': [source_lon, receiver_lon],
    'depth_km': [source_depth_km, source_depth_km],
})
pl.show()